# Image Segmentation using the idr0021 dataset

This notebook uses the idr0021 dataset (original: [idr0021-lawo-pericentriolarmaterial/experimentA](http://idr.openmicroscopy.org/webclient/?show=project-51)) and tries to reproduce some of the analysis published in ['Subdiffraction imaging of centrosomes reveals higher-order organizational features of pericentriolar material'](https://doi.org/10.1038/ncb2591). In particular we're trying to get a figure similar to [Figure 1](https://www.nature.com/articles/ncb2591/figures/1) of the article. 

Each user has a partial copy of the dataset ('Lab1' / YOUR USER / Project: 'idr0021'). Some of the images are also part of the dataset 'R-dataset' which you can use for practicing the segmentation and annotation steps.

**Exercise:** Go to https://outreach.openmicroscopy.org , log in as your given user, and find the dataset ('Lab1' / YOUR USER / Project: 'R-dataset'). Leave the window open, as you will use OMERO.web to check the results as we're going along.

## How to use this notebook

The grey blocks with the In\[ \] notation to the left are code blocks. Click on a block to select it and click the play button in the toolbar to execute the block of code (alternatively hold SHIFT key and hit RETURN). The notation will change to In\[*\] to indicate that this code is currently running. The output is displayed below the block. Wait for it to finish. When the execution is finished it will change to ln\[1\] showing that this was the first block of code you ran (ln\[2\], ln\[3\], ... for the next executed code blocks). 

As we go through the notebook you're expected to run the code blocks in order (unless explicitely mentioned otherwise).

Here's an example:

In [ ]:
message("Just wait for a bit...")
Sys.sleep(3)
message("Done.")

You can change the code and run the block again.

**Exercise:** Change the message which is printed and run it again.

## Let's get started...

Load the necessary [romero-gateway](https://github.com/ome/rOMERO-gateway) and [EBImage](https://bioconductor.org/packages/release/bioc/html/EBImage.html) library. We're currently using romero-gateway 0.4.3, EBImage 4.13.0 with R 3.4.1 . 

In [ ]:
library(romero.gateway)
library(EBImage, warn.conflicts = FALSE)

## Log into the OMERO server

In [ ]:
user_name = readline('Username: ')
user_password <- getPass::getPass('OMERO password: ')
server <- OMEROServer(host = 'outreach.openmicroscopy.org', username=user_name, password=user_password, port= as.integer(4064))
server <- connect(server)
paste('Successfully logged in as', server@user$getUserName())

## Load and display an image

Load an image from the example dataset 'R-dataset' (these images are part of the original 'CDK5RAP2-C' dataset).

**Exercise:** Go to the image 'siControl_N20_Cep215_I_20110411_Mon-1509_0_SIR_PRJ.dv' in your 'R-dataset', find the 'Image ID' and copy/paste it below. You can use a different image of the dataset if you like, but I recommend that one because it nicely shows the toroid centrioles.

In [ ]:
imageId <- REPLACE_WITH_IMAGE_ID # An image from R-dataset
image <- loadObject(server, "ImageData", imageId)

Load the pixel values of the second channel (the CDK5RAP2-C channel) and display the image

In [ ]:
pixels <- getPixelValues(image, 1, 1, 2) # Second channel of the first z plane of the first time point
                                         # (the images only have one z/t plane anyway)
ebi <- EBImage::Image(data = pixels, colormode = 'Grayscale')
ebi <- normalize(ebi)
EBImage::display(ebi)

## Image segmentation

### Visual

In [ ]:
threshImg <- thresh(ebi, w=15, h=15, offset=0.1)
threshImg <- medianFilter(threshImg, size=3)
threshImg <- fillHull(threshImg)          
threshImg <- bwlabel(threshImg)
EBImage::display(colorLabels(threshImg))

**Exercise:** Modify the parameters of the thresh function to get a feeling for how the parameters affect the segmentation, or even plug in other methods into the segmentation workflow to improve the result, see [EBImage documentation](https://www.rdocumentation.org/packages/EBImage/versions/4.14.2)

### Compute the features (measurements)

In [ ]:
shapes = computeFeatures.shape(threshImg)
moments = computeFeatures.moment(threshImg)
features <- merge(shapes, moments, by=0, all=TRUE)
features

## Save the results back to OMERO

### Create ROIs

We can create an Ellipse for each object taking the x,y coordinates (m.cx and m.cy), the radii (this is the m.majoraxis and calculate the 'minor' axis from 'major' axis and the eccentricity (see [documentation](https://www.rdocumentation.org/packages/EBImage/versions/4.14.2/topics/computeFeatures) of the computeFeatures method)) and the angle theta by which the Ellipse might be rotated.

In order to be able to re-use this code later, we define a function for it:

In [ ]:
#' Create ROIs from a features table
#'
#' @param features The shape and moments generated by EBImage computeFeatures
#' @return A dataframe specifying the x, y, rx, ry, w, h, theta and text parameters of the ROIs
createROIs <- function(features) {
    rois <- data.frame(x = c(0), y = c(0), rx = c(0), ry = c(0), w = c(0), h = c(0), theta = c(0), text = c('remove'), stringsAsFactors = FALSE)
    for (index in 1:length(features[,1])) {
        x <- features[index,8]
        y <- features[index,9]
        r1 <- features[index,10]
        ecc <- features[index,11]
        r2 <- sqrt(r1^2 * (1 - ecc^2))
        theta <- features[index,12]
        rois <- rbind(rois, c(x, y, r1, r2, NA, NA, -theta, as.character(index)))
    }
    rois <- rois[-1,]
    rownames(rois) <- c()
    return(rois)
}

In [ ]:
rois <- createROIs(features)
rois

Save the ROIs back to OMERO

In [ ]:
addROIs(image, coords = rois)

**Exercise:** Open the image in OMERO.web and compare the ROIs with the segmented image.

### Store the features table

Attach the whole dataframe to the image:

In [ ]:
invisible(attachDataframe(image, features, "ROI features")) # the invisible just suppresses some unneccessary output

The dataframe is now attached to the image as an [HDF](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) table. You can download and open it with software like HDF Compass, load it into python scripts using 'h5py' library, etc. or simply load it directly as R dataframe again.

Alternatively attach it as CSV file:

In [ ]:
csvFile <- "/tmp/ROI_features.csv"
write.csv(features, file = csvFile)
fileannotation <- attachFile(image, csvFile)

**Exercise:** Find the attached CSV file and open it in Excel

## Automate

Task: Segment each image of a dataset, create an ROI for each of the objects, and summarize the results as dataframe, only taking the features of the largest ROI of an image into account.

Note: We need to be able to specify the channel which is analyzed by its name.

We put the all the pieces together and wrap them up in a function:

In [ ]:
#' Performs an image segmentation to find the largest ROI of the image
#' and returns some features of interest (area, perimeter and diameter).
#'
#' @param image The image to segement
#' @param channelName The channel to be taken into account
#' @param df The dataframe to add the features to (channelName, imageName, ImageID, ROIINdex, area, perimeter, diameter)
#' @param saveROIs Flag if ROIs should be created and attached to the image (default: FALSE)
#' @return The dataframe with the features
analyzeImage <- function(image, channelName, df, saveROIs = FALSE) {
    # Find the channel index
    chnames <- getChannelNames(image)
    chindex <- match(channelName, chnames, nomatch = 0)
    if (chindex == 0) {
      message (paste("Could not resolve channel name, skipping ", image@dataobject$getId()))
      return(df)
    }
    
    # Load the pixels
    pixels <- getPixelValues(image, 1, 1, chindex)
    ebi <- EBImage::Image(data = pixels, colormode = 'Grayscale')
    ebi <- normalize(ebi)
     
    # this is our segmentation workflow from above
    threshImg <- thresh(ebi, w=15, h=15, offset=0.1)
    threshImg <- medianFilter(threshImg, size=3)
    threshImg <- fillHull(threshImg)          
    threshImg <- bwlabel(threshImg)
    
    # Calculate the features
    shapes = suppressMessages(computeFeatures.shape(threshImg))
    moments = suppressMessages(computeFeatures.moment(threshImg))
    features <- merge(shapes, moments, by=0, all=TRUE)
    
    if (length(features[,1])>1) {
        # Add the ROIs to the image
        rois <- createROIs(features)
        if (saveROIs)
            addROIs(image, coords = rois)
        
        # Add the interesting properties (area, perimeter and diameter)
        # of the largest ROI together with channel name, image name, image id 
        # and roi index to the dataframe
        features <- features[order(-features[,2]),]
        diameter <- features[1,4]*2
        df <- rbind(df, c(channelName, image@dataobject$getName(), image@dataobject$getId(), features[1,1], features[1,2], features[1,3], diameter))
    }
    return(df)
}

In [ ]:
datasetId <- REPLACE_WITH_DATASET_ID # Insert the ID of your R-dataset here!

channelName <- 'CDK5RAP2-C'

dataset <- loadObject(server, "DatasetData", datasetId)

# Keep the channel name, image name, image id, area, perimeter, and diameter of the largest ROIs
result <- data.frame(Channel = c('remove'), ImageName = c('remove'), Image = c(0), ROIIndex = c(0), Area = c(0), Perimeter = c(0), Diameter = c(0), stringsAsFactors = FALSE)

images <- getImages(dataset)
for (image in images) {
    result <- tryCatch({
        analyzeImage(image, channelName, result, saveROIs = TRUE)
    }, warning = function(war) {
        message(paste("WARNING:  ", image@dataobject$getId(),war))
        return(result)
    }, error = function(err) {
        message(paste("ERROR:  ", image@dataobject$getId() ,err))
        return(result)
    }, finally = {
    })
}

result <- result[-1,]
rownames(result) <- c()

# set the correct datatypes
result$Channel <- as.factor(result$Channel)
result$Area <- as.numeric(result$Area)
result$Perimeter <- as.numeric(result$Perimeter)
result$Diameter <- as.numeric(result$Diameter)

result

**Exercise:** Check the ROIs in OMERO.web

**Exercise:** The measurements (Area, Perimeter and Diameter) are currently showing pixels. Find out the pixel size in nanometer and apply it accordingly.

## Analyze

### Run segmentation over the whole project

**Do not run this, FYI only!**

The next code block does the same what we did before but across the whole idr0021 project (taking into account that the channel name which has to be analyzed is equal to the dataset name). This can take a while, therefore we skip this step and load the results from the attached dataframe, see next step.

In [ ]:
projectId <- ---

project <- loadObject(server, "ProjectData", projectId)
datasets <- getDatasets(project)

result <- data.frame(Channel = c('remove'), ImageName = c('remove'), Image = c(0), ROIIndex = c(0), Area = c(0), Perimeter = c(0), Diameter = c(0), stringsAsFactors = FALSE)

for (dataset in datasets) {
    dsname <- dataset@dataobject$getName()
    if (startsWith(dsname, "CEP120")) # CEP120 data is represented by two datasets (both start with CEP120)
        dsname <- "CEP120"
    images <- getImages(dataset)
    for (image in images) {
        result <- tryCatch({
            analyzeImage(image, dsname, result, saveROIs = TRUE)
        }, warning = function(war) {
            message(paste("WARNING:  ", image@dataobject$getId(),war))
            return(result)
        }, error = function(err) {
            message(paste("ERROR:  ", image@dataobject$getId() ,err))
            return(result)
        }, finally = {
        })
    }
}

# remove the first row and row names
result <- result[-1,]
rownames(result) <- c()

# set the correct datatypes
result$Channel <- as.factor(result$Channel)
result$Area <- as.numeric(result$Area)
result$Perimeter <- as.numeric(result$Perimeter)
result$Diameter <- as.numeric(result$Diameter)
result$Image <- as.integer(result$Image)

# set the correct pixel size
pxSizeInNM <- 40
result$Area <- result$Area * pxSizeInNM * pxSizeInNM
result$Perimeter <- result$Perimeter * pxSizeInNM
result$Diameter <- result$Diameter * pxSizeInNM

# Rename the Channel column to Dataset (for parade)
colnames(result)[1] <- "Dataset"
result$Image <- as.integer(result$Image)

# attach the dataframe to the project
invisible(attachDataframe(project, result, "Summary from R"))

# Change the namespace (for parade) (workaround as this is not yet
# implemented in the R gateway itself)
dataframes <- availableDataframes(project)
faid <- dataframes$AnnotationID[1]
fa <- loadObject(server, "omero.gateway.model.FileAnnotationData", as.integer(faid))
fa@dataobject$setNameSpace("openmicroscopy.org/omero/bulk_annotations")
gateway <- getGateway(server)
ctx <- getContext(server)
dm <- gateway$getFacility(DataManagerFacility$class)
fa <- dm$saveAndReturnObject(ctx, fa@dataobject)

### Load the segmentation results

Instead of running the segmentation over the whole idr0021 project yourself, you'll use the summary table of trainer-1.

**Exercise:** In OMERO.web got to Lab1 / trainer-1 and find out the ID of the idr0021 project.

In [ ]:
projectId <- REPLACE_WITH_PROJEcT_ID # This is the ID of trainer-1's idr0021 project
project <- loadObject(server, "ProjectData", projectId)
dataframes <- availableDataframes(project)
print(dataframes)

In [ ]:
# Load the dataframe
dfID <- dataframes$ID[1]
centrioles <- loadDataframe(project, dfID)

# Make sure the data types are correct
centrioles$Dataset <- as.factor(centrioles$Dataset)
centrioles$Diameter <- as.numeric(centrioles$Diameter)
centrioles

### Plot the data

In [ ]:
ag <-aggregate(centrioles$Diameter ~ centrioles$Dataset, centrioles, median)
ordered <- factor(centrioles$Dataset, levels=ag[order(ag$`centrioles$Diameter`), 'centrioles$Dataset'])

In [ ]:
plot(centrioles$Diameter ~ ordered, ylab='Diameter (nm)', xlab="Protein", cex.axis=0.5)

**Exercise:** Find the outlier in OMERO.web (using 'parade') and check what might have gone wrong there.

### Compute some statistics

In [ ]:
# One-way analysis of variance:
fit <- aov(centrioles$Diameter ~ centrioles$Dataset)
summary(fit)

In [ ]:
# Two-sample Wilcoxon test ('Mann-Whitney') of all pairwise combinations:
combins <- combn(levels(centrioles$Dataset), 2)
params_list <- split(as.vector(combins), rep(1:ncol(combins), each = nrow(combins)))
testResults <- data.frame()
for (param in params_list) {
  testdf <- subset(centrioles, centrioles$Dataset %in% param)
  pval <- wilcox.test(formula = Diameter ~ Dataset, data = testdf)$p.value
  testResults <- rbind(testResults, data.frame(Protein_1=param[1], Protein_2=param[2], p_value=pval))
}
testResults

In [ ]:
disconnect(server)